In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [3]:
!mkdir pages

In [12]:
files = glob('threads/*.json')
len(files)

164

In [13]:
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)

    for d in data:
        if d[-1] > 1:
            url = d[0]
            for i in range(2, d[-1] + 1, 1):
                urls.append(f'{url}/page-{i}')

100%|█████████████████████████████████████████████████████████████████████████████████████████| 164/164 [00:00<00:00, 2133.13it/s]


In [14]:
urls = sorted(list(set(urls)))
len(urls)

33093

In [15]:
urls[0]

'https://www.skyscrapercity.com/threads/%E2%80%A2-cityscapes-list-of-towns-cities-%E2%80%A2.935620/page-2'

In [17]:
def get_url(url):
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    td = soup.find_all('div', {'class': 'bbWrapper'})
    td = [td_.text.strip() for td_ in td]
    td = [td_ for td_ in td if len(td_)]
    return td

In [18]:
max_worker = 20
for i in tqdm(range(0, len(urls), max_worker)):
    
    filename = os.path.join('pages', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
    
    results = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_url, url): url for url in urls[i: i + max_worker]}

        for future in as_completed(futures):
            results.append(future.result())
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)
        
    try:
        del results
        del futures
    except:
        pass

100%|█████████████████████████████████████████████████████████████████████████████████████████| 1655/1655 [52:00<00:00,  1.89s/it]


In [28]:
files = glob('threads/*.json')
len(files)

164

In [29]:
texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d[1]:
            texts.append(d_)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 164/164 [00:00<00:00, 2892.54it/s]


In [30]:
files = glob('pages/*.json')
len(files)

1655

In [31]:
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d:
            texts.append(d_)

100%|███████████████████████████████████████████████████████████████████████████████████████| 1655/1655 [00:00<00:00, 1660.52it/s]


In [32]:
texts = list(set(texts))
len(texts)

633366

In [33]:
with open('everything.jsonl', 'w') as fopen:
    for t in tqdm(texts):
        if len(t):
            fopen.write(f'{json.dumps(t)}\n')

100%|█████████████████████████████████████████████████████████████████████████████████| 633366/633366 [00:01<00:00, 481901.29it/s]
